## Exploring the CAJ AER paper.

In [1]:
import json, glob
from caj_tools import *

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Get all the graphs -- they're all set.

In [2]:
graphs = {}
for f in glob.glob("f*/*.json"):
    with open(f) as i:
        graphs[f.replace(".json", "")] = json.load(i)

This is what the graphs look like.

In [3]:
graphs["f1/a"]

[{'id': 1, 'nei': [2]}, {'id': 2, 'nei': [1, 3]}, {'id': 3, 'nei': [2]}]

### In Figure 1, agent 1 benefits from agent 3 in the long run (s₁(a) > s₁(b)).  But for s<sub>t-1</sub> = (0, 1, 0) as given, they are anticorrelated.

In [4]:
iterate(graphs["f1/a"], N = 20000000)
iterate(graphs["f1/b"], N = 20000000)

In [5]:
unemployment(graphs["f1/a"][0]["hist"])
unemployment(graphs["f1/b"][0]["hist"])

0.078264

0.07830250000000005

In [6]:
s1_hist = graphs["f1/a"][0]["hist"]
s2_hist = graphs["f1/a"][1]["hist"]
s3_hist = graphs["f1/a"][2]["hist"]

s1_cond = [x for xi, x in enumerate(graphs["f1/a"][0]["hist"])
           if xi and s1_hist[xi-1] == 0 and s2_hist[xi-1] == 1 and s3_hist[xi-1] == 0]

s3_cond = [x for xi, x in enumerate(graphs["f1/a"][2]["hist"])
           if xi and s1_hist[xi-1] == 0 and s2_hist[xi-1] == 1 and s3_hist[xi-1] == 0]

correlation(s1_cond, s3_cond)

-0.01707960682112768

### In Figure 2 A-D, the employment rates are 0.132, 0.083, 0.063, and 0.050.

In [7]:
iterate(graphs["f2/a"], N = 10000000)
iterate(graphs["f2/b"], N = 10000000)
iterate(graphs["f2/c"], N = 10000000)
iterate(graphs["f2/d"], N = 10000000)

In [8]:
unemployment(graphs["f2/a"][0]["hist"])
unemployment(graphs["f2/b"][0]["hist"])
unemployment(graphs["f2/c"][0]["hist"])
unemployment(graphs["f2/d"][0]["hist"])

0.13192459999999995

0.07888620000000002

0.05663830000000003

0.04657560000000005

No, but their footnotes 8 and 9 are not encouraging:

_The numbers for more than one agent are obtained from simulations in Maple􏰥. We simulate the economy over a large number of periods (hundreds of thousands) and calculate observed unemployment averages and correlations. The programs are available upon request from the authors. The correlation numbers are only moderately accurate, even after several hundred thousand periods._

_In some cases, the correlations are indistinguishable to the accuracy of our simulations._

If I use as few iterations, I can reproduce their numbers.

### The Corr(s₁ s₂) are --, 0.041, 0.025, and 0.025.

In [9]:
correlation(graphs["f2/a"][0]["hist"], graphs["f2/a"][1]["hist"])
correlation(graphs["f2/b"][0]["hist"], graphs["f2/b"][1]["hist"])
correlation(graphs["f2/c"][0]["hist"], graphs["f2/c"][1]["hist"])
correlation(graphs["f2/d"][0]["hist"], graphs["f2/d"][1]["hist"])

0.00019436092423589482

0.053055982755748125

0.019752073072236313

0.026060963830738245

### Figure 3: The unemployment rate for A is 6% and for B is 3%.

In [10]:
iterate(graphs["f3/a"], N = 10000000)
iterate(graphs["f3/b"], N = 1000000)

unemployment(graphs["f3/a"][0]["hist"])
unemployment(graphs["f3/b"][0]["hist"])

0.05657469999999998

0.029317000000000037

### The correlations fall with the length of the path.

In [11]:
correlation(graphs["f3/a"][0]["hist"], graphs["f3/a"][1]["hist"])
correlation(graphs["f3/a"][0]["hist"], graphs["f3/a"][2]["hist"])
correlation(graphs["f3/a"][0]["hist"], graphs["f3/a"][3]["hist"])
correlation(graphs["f3/a"][0]["hist"], graphs["f3/a"][4]["hist"])

0.01960437595478154

0.008457364512102812

0.0008520246444672274

-0.0008828857380046936

### Figure 4: The employment rates for types 1, 2, and 3 are 0.047, 0.048, and 0.050.

In [12]:
iterate(graphs["f4/a"], N = 10000000)

unemp_rates = {}
for node in range(10):
    unemp_rates[node+1] = unemployment(graphs["f4/a"][node]["hist"])
    print("{:d} :: {:.4f}".format(node+1, unemp_rates[node+1]))

(unemp_rates[1] + unemp_rates[6])/2
(unemp_rates[2] + unemp_rates[5] + unemp_rates[7] + unemp_rates[10])/4
(unemp_rates[3] + unemp_rates[4] + unemp_rates[8] + unemp_rates[9]) /4

1 :: 0.0453
2 :: 0.0458
3 :: 0.0459
4 :: 0.0459
5 :: 0.0459
6 :: 0.0453
7 :: 0.0457
8 :: 0.0457
9 :: 0.0462
10 :: 0.0460


0.04528949999999998

0.045841875000000004

0.04593382500000001

### The average path length matters for unemployment

We can get the path lengths right, but the employment rates are slightly lower than they quote, and only very slightly different.

In [13]:
avg_path_length(graphs["f5/a"]), avg_path_length(graphs["f5/b"])

(1.5714285714285714, 1.7857142857142856)

In [14]:
iterate(graphs["f5/a"], N = 5000000)
iterate(graphs["f5/b"], N = 5000000)

sum([unemployment(graphs["f5/a"][n]["hist"]) for n in range(8)])/8
sum([unemployment(graphs["f5/b"][n]["hist"]) for n in range(8)])/8

0.04544277499999999

0.045620700000000014

### §III, Fig. 6: for connected agents, expectations of next-round employment fall with length of unemployment duration, because duration is indicative of peers current status.

In [15]:
def conditional_prob_periods(hist, periods):
    
    cond = []
    for xi in range(periods, len(hist)):
        if not any([hist[xpi] for xpi in range(xi-periods, xi)]):
            cond.append(hist[xi])
            
    return sum(cond)/len(cond)

conditional_prob_periods(graphs["f2/b"][0]["hist"], 1)
conditional_prob_periods(graphs["f2/b"][0]["hist"], 2)
conditional_prob_periods(graphs["f2/b"][0]["hist"], 10)
conditional_prob_periods(graphs["f2/b"][0]["hist"], 20)

conditional_prob_periods(graphs["f2/d"][0]["hist"], 1)
conditional_prob_periods(graphs["f2/d"][0]["hist"], 2)
conditional_prob_periods(graphs["f2/d"][0]["hist"], 10)
conditional_prob_periods(graphs["f2/d"][0]["hist"], 20)

0.17561499983520565

0.1746262643658719

0.16679054147053157

0.16446764091858038

0.30831594225302517

0.302927774121854

0.2752160687557337

0.2405857740585774